In [1]:
import utils
import numpy as np
import edf
from time import time
import pickle
import os

train_data, trcnt = utils.load_data_onechar('data/ptb.train.txt')
valid_data, vacnt = utils.load_data_onechar('data/ptb.valid.txt')
test_data, tecnt = utils.load_data_onechar('data/ptb.test.txt')

# HINT
Let's see inside of the data. train_data is list of char index.
$$
    \forall i \ \ \texttt{train_data[i]} \in Z^{|s|}
$$
where $s$ is a sequence of charactors. And
$$
    s \in \{c \in C\}
$$
$$
    C \in \{a,b,c,...,` `,`\{`, `\}`\}
$$
where "{" and "}" are special tags for head and end of sentences.

( NOTE ) we do NOT use pos-tag or any language information on PTB except sequence of charactors at all! 
and <font color="red"> you might realize that the word $"<UNK>"$ makes strange charactor sequecne $"<,U,N,K,>"$. Let's think there is a word with $"<UNK>"$ in this world!  lol
</font>

In [68]:
#===========================
#          HINT
#===========================
#Charactor mapings
chars = [None]*len(utils.vocab.keys())
for k in utils.vocab.keys(): chars[utils.vocab[k]]=k
print "Charactor mapping : "
print chars

#inside of data
samples = train_data[1000:1005]
print "samples : "
for i,s in enumerate(samples):
    print "indeces : ",s
    print "real chars : ",
    for c in s:
        print chars[c],
    print

Charactor mapping : 
['@', u'{', u'a', u'e', u'r', u' ', u'b', u'n', u'k', u'o', u't', u'l', u'i', u'z', u'c', u'w', u'y', u'u', u's', u'f', u'm', u'g', u'h', u'd', u'-', u'q', u'p', u'x', u'}', u'<', u'>', u'j', u'v', u'.', u"'", u'1', u'9', u'5', u'0', u'&', u'$', u'3', u'2', u'4', u'8', u'6', u'7', u'#', u'\\', u'/', u'*']
samples : 
indeces :  [ 1 21  9 32  3  4  7 20  3  7 10  5 14  9  7 10  4  2 14 10  9  4 18 28]
real chars :  { g o v e r n m e n t   c o n t r a c t o r s }
indeces :  [ 1 10 22  2 10  5 12 18  5 19  9  4  5 10 22  3  5 19 17 10 17  4  3 28]
real chars :  { t h a t   i s   f o r   t h e   f u t u r e }
indeces :  [ 1 18 10 12 11 11  5 14  2 17 10 12  9  7  5 12 18  5 29 17  7  8 30 28]
real chars :  { s t i l l   c a u t i o n   i s   < u n k > }
indeces :  [ 1 18 10  9 14  8 18  5 32  9 11 17 20  3  5  7  5 18 22  2  4  3 18 28]
real chars :  { s t o c k s   v o l u m e   n   s h a r e s }
indeces :  [ 1 22  3  5 15  9 17 11 23  5  7 34 10  5  3 11  2  6  9  4  

# HINT
## Probabilities of LSTM model
Here let's see what we want to code. We need LSTM which can predict next charactor from the previous sequece, i.e.,
$$
    {\rm LSTM} : C^{t-1} \rightarrow C
$$
thus LSTM gives probability distribution over $C$,
$$
    P_{{\rm LSTM}}( s_t | s_{<t} ) = softmax(V_{[s_t]}h_t)   
$$
where $V_{[s_t]}$ is another parameter of vector for word $s_t$, $h_t$ is $t$-th output from the LSTM, $s_i$ is $i$-th charactor in the sentence $s$ and $s_{<t}$ is charactors before $t$.

Then log likelihood of a sentence is 
$$
    \log \mathcal{L} = \sum_{0<=t<T} \log P_{{\rm LSTM}}(s_t| s_{<t})
$$
<font color="red"> NOTE) we also want to predict the first charactor of a sentence which is always "{" and the last one which is "}".</font>


## Implementation tricks
We train the LSTM with dynamically constructing RNN sequence for each batch (we do not stick to a statistic computational graph anymore), i.e.,

FOR each batch, $b$ :  
| ${\tt loss, score = buildModel()}$  
| ${\tt edf.Forward()}$  
| ${\tt edf.Backward(loss)}$  
| ${\tt edf.GradClip(10)}$  
| ${\tt edf.SGD(eta)}$  
END

Now ${\tt buildModel()}$ is

${\tt buildModel()}$  
| Global_var : ${\tt inp}$, ${\tt C2V}$, ${\tt hidden\_dim}$, $LSTM\_PARAS$   
| ${\tt loss\_list = []}$  #List of logL at each time $t$   
| ${\tt score= []}$       #List of probabilities at each time $t$   
| $\tt FOR\ t<T$ :  
| | chain LSTM_CELL  
| | compute logLoss,$\tt l$, and prob,$\tt p$, at time $\tt t$  
| | ${\tt loss\_list += [l]}$  
| | ${\tt score += [p]}$   
| $\tt END$   
| ${\tt loss = average(loss\_list)}$   
| ${\tt return\ loss, score}$   
$\tt END$  

Again $\tt score$ is a list of probabilities on each time step $t$ and $\tt loss$ is a log likelihood of the batch.


## Other things you might concern 
### computational graph
Now computational graph is not static, we need to initialize everytime.

### initial state of LSTM cell 
We can think several reasonable implementation but we employed just zero vector.

### masks and paddings
We train model with mini-batch but LSTM chain has a fix size for all sentences in a batch. We need to remove output from padded part.


In [88]:
hidden_dim = 200
n_vocab = utils.n_vocab
batch = 50
parameters = []
model = 'model_LSTM.pkl'
eta = 0.5
decay = 0.9

#debug
batch=4
hidden_dim=10

inp = edf.Value()
np.random.seed(0)

edf.params = []

# LSTM parameters
C2V = edf.Param(edf.xavier((n_vocab, hidden_dim)))

# forget gate
Wf = edf.Param(edf.xavier((2*hidden_dim, hidden_dim)))
bf = edf.Param(np.zeros((hidden_dim)))
# input gate
Wi = edf.Param(edf.xavier((2*hidden_dim, hidden_dim)))
bi = edf.Param(np.zeros((hidden_dim)))
# carry cell
Wc = edf.Param(edf.xavier((2*hidden_dim, hidden_dim)))
bc = edf.Param(np.zeros((hidden_dim)))
# output cell
Wo = edf.Param(edf.xavier((2*hidden_dim, hidden_dim)))
bo = edf.Param(np.zeros((hidden_dim)))
    
V = edf.Param(edf.xavier((hidden_dim, n_vocab)))
# for sake of saving
parameters.extend([C2V, Wf, bf, Wi, bi, Wc, bc, Wo, bo, V])


# load the trained model if exist
if os.path.exists(model):
    with open(model, 'rb') as f:
        p_value = pickle.load(f)
        idx = 0
        for p in p_value:
            parameters[idx].value = p
            idx += 1
                    
# LSTM cell
def LSTMCell(xt, h, c):
    
    f = edf.Sigmoid(edf.Add(edf.VDot(edf.ConCat(xt, h), Wf), bf))
    i = edf.Sigmoid(edf.Add(edf.VDot(edf.ConCat(xt, h), Wi), bi))
    o = edf.Sigmoid(edf.Add(edf.VDot(edf.ConCat(xt, h), Wo), bo))
    c_hat = edf.Tanh(edf.Add(edf.VDot(edf.ConCat(xt, h), Wc), bc))
    c_next = edf.Add(edf.Mul(f, c), edf.Mul(i, c_hat))
    h_next = edf.Mul(o, edf.Tanh(c_next))
            
    return h_next, c_next

# build the model given the input inp, it shoudl return loss and prob
def BuildModel():
    '''
    Global vars:
        inp : Z^{batch_size x seq_size}
    '''
    
    #initialize CP
    edf.components = []

    B = inp.value.shape[0] #batch_size
    T = inp.value.shape[1] #length of sequence (max len(s))
    h = edf.Value(np.zeros((B, hidden_dim))) #initial state of the cell
    c = edf.Value(np.zeros((B, hidden_dim)))
    
    score = []
    
    for t in range(T-1):
 
        wordvec = edf.Embed(edf.Value(inp.value[:,t]), C2V) #word embedings on time t R^{batch_size x hidden_dim}
        xt = edf.Reshape(wordvec, [-1, hidden_dim])         #!!UNNECESSARY RESHAPE!!
        h_next, c_next = LSTMCell(xt, h, c)
        p = edf.SoftMax(edf.VDot(h_next, V))
        l = edf.LogLoss(edf.Aref(p, edf.Value(inp.value[:,t+1]))) #R^{batch_size}
        logloss = edf.Reshape(l, (B, 1))   #R^{batch_size x 1}
        
        if t == 0:
            loss = logloss
        else:
            loss = edf.ConCat(loss, logloss) #R^{batch_size x T}
            
        score.append(p)    
        h = h_next
        c = c_next
    
    masks = np.zeros((B, T-1), dtype = np.int32)
    masks[inp.value[:,1:] != 0] = 1
    loss = edf.MeanwithMask(loss, edf.Value(masks)) 
    
    return loss, score,[l, logloss]
    
    
# calculate the perplexity         
def CalPerp(score):
    
    prob = [p.value for p in score]
    prob = np.transpose(np.stack(prob, axis = 0),(1,0,2))
    
    B = prob.shape[0]
    T = prob.shape[1]
    V = prob.shape[2]
    
    masks = np.zeros((B, T), dtype=np.int32)
    masks[inp.value[:,1:] != 0] = 1
    
    prob = prob.reshape(-1)
    idx = np.int32(inp.value[:,1:].reshape(-1))
    outer_dim = len(idx)
    inner_dim = len(prob)/outer_dim
    pick = np.int32(np.array(range(outer_dim))*inner_dim + idx)
    prob = prob[pick].reshape(B, T)
        
    return -np.sum(np.log(prob[np.nonzero(prob*masks)]))


# predict the sequence
def Predict(max_step, prefix):
   
    edf.components = []

    T = max_step       
    h = edf.Value(np.zeros((1, hidden_dim))) 
    c = edf.Value(np.zeros((1, hidden_dim))) 
    
    prediction = []

    for t in range(T):
   
        if t < len(prefix):
            pred = edf.Value(prefix[t])
            prediction.append(pred)              
        else:
            prediction.append(pred)

        wordvec = edf.Embed(pred, C2V)
        xt = edf.Reshape(wordvec, [-1, hidden_dim])
        h_next,c_next = LSTMCell(xt, h, c)
        p = edf.SoftMax(edf.VDot(h_next, V))
        pred = edf.ArgMax(p)
        h = h_next
        c = c_next   
            
    edf.Forward()
    
    idx = [pred.value for pred in prediction]
    stop_idx = utils.to_index('}')
    
    if stop_idx in idx:
        return idx[0:idx.index(stop_idx)+1]
    else:
        return idx

def Eval(data, cnt):
    
    perp = 0.
    avg_loss = 0.
    test_batches = range(0, len(data), batch)
    test_minbatches = [data[idx:idx+batch] for idx in test_batches]
    
    for minbatch in test_minbatches:
        
        x_padded = utils.make_mask(minbatch)
        inp.set(x_padded)
        loss, score,_ = BuildModel()
        edf.Forward()
        avg_loss += loss.value
        perp += CalPerp(score)
           
    perp = np.exp(perp/cnt)
    avg_loss /= len(test_batches)
    return perp, avg_loss


############################################### training loop #####################################################

batches = range(0, len(train_data), batch)
minbatches = [train_data[idx:idx+batch] for idx in batches]

epoch = 30

# initial Perplexity and loss
perp, loss = Eval(valid_data, vacnt)
print("Initial: Perplexity: %0.5f Avg loss = %0.5f" % (perp, loss))    
best_loss = loss
prefix = 'the agreements bring'  
generation = Predict(400, utils.to_idxs(prefix))
print("Initial generated sentence ")
print (utils.to_string(generation))
    
    
for ep in range(epoch):

    perm = np.random.permutation(len(minbatches)).tolist() 
    stime=time()
    
    print ep,"-th epoch"
    for k in range(len(minbatches)):
        
        minbatch = minbatches[perm[k]]
        x_padded = utils.make_mask(minbatch)
        inp.set(x_padded)
        loss, score,debug = BuildModel()
        edf.Forward()
        for v in debug: print v.value
        edf.Backward(loss)
        edf.GradClip(10)
        edf.SGD(eta)
       
    duration = (time() - stime)/60.
    
    perp, loss = Eval(valid_data, vacnt)
    print("Epoch %d: Perplexity: %0.5f Avg loss = %0.5f [%.3f mins]" % (ep, perp, loss, duration))
    
    # generate some text given the prefix and trained model
    prefix = 'the agreements bring'  
    generation = Predict(400, utils.to_idxs(prefix))
    print("Epoch %d: generated sentence " % ep)
    print (utils.to_string(generation)) 

    if loss < best_loss:
        
        best_loss = loss
        # save the model
        f = open(model, 'wb')
        p_value = []
        for p in parameters:
            p_value.append(p.value)
        pickle.dump(p_value, f)
        
    else:
        
        # load the last best model and decay the learning rate
        eta *= decay
        with open(model, 'rb') as f:
            p_value = pickle.load(f)
            idx = 0
            for p in p_value:
                parameters[idx].value = p
                idx += 1

Initial: Perplexity: 49.37356 Avg loss = 3.93269
Initial generated sentence 
the agreements bringun1fffff{myeiieieieie<zn#lt<<.2#ll66{6y{iffff{myeieieieie<zn#lt<<.2#ll66{6y{iffff{myeieieieie<zn#lt<<.2#ll66{6y{iffff{myeieieieie<zn#lt<<.2#ll66{6y{iffff{myeieieieie<zn#lt<<.2#ll66{6y{iffff{myeieieieie<zn#lt<<.2#ll66{6y{iffff{myeieieieie<zn#lt<<.2#ll66{6y{iffff{myeieieieie<zn#lt<<.2#ll66{6y{iffff{myeieieieie<zn#lt<<.2#ll66{6y{iffff{myeieieieie<zn#lt<<.2#ll66{6y{iffff{myeieieiei
0 -th epoch
[ 3.90744042  3.98190022  3.95454621  3.89998221]
[[ 3.90744042]
 [ 3.98190022]
 [ 3.95454621]
 [ 3.89998221]]
[ 3.90480028  3.93716897  3.91379222  3.91539979]
[[ 3.90480028]
 [ 3.93716897]
 [ 3.91379222]
 [ 3.91539979]]
[ 3.94551179  3.95401026  3.9995839   3.99814916]
[[ 3.94551179]
 [ 3.95401026]
 [ 3.9995839 ]
 [ 3.99814916]]
[ 3.9420608   3.94391443  3.94255445  3.92786874]
[[ 3.9420608 ]
 [ 3.94391443]
 [ 3.94255445]
 [ 3.92786874]]
[ 3.92362923  3.92762314  3.93848426  3.99557313]
[[ 3.92362923]
 

KeyboardInterrupt: 

In [80]:
hidden_dim = 200
n_vocab = utils.n_vocab
batch = 50
parameters = []
model = 'model_GRU.pkl'
eta = 0.5
decay = 0.9


inp = edf.Value()
np.random.seed(0)

# GRU parameters
edf.params = []
C2V = edf.Param(edf.xavier((n_vocab, hidden_dim)))
Wz = edf.Param(edf.xavier((2*hidden_dim, hidden_dim)))
Wr = edf.Param(edf.xavier((2*hidden_dim, hidden_dim)))
W = edf.Param(edf.xavier((2*hidden_dim, hidden_dim)))
V = edf.Param(edf.xavier((hidden_dim, n_vocab)))

# for sake of saving
parameters.extend([C2V, Wz, Wr, W, V])

# load the trained model if exist
if os.path.exists(model):
    with open(model, 'rb') as f:
        p_value = pickle.load(f)
        idx = 0
        for p in p_value:
            parameters[idx].value = p
            idx += 1
                    
# GRU cell                
def GRUCell(xt, h):
    
    z = edf.Sigmoid(edf.VDot(edf.ConCat(xt, h), Wz))
    r = edf.Sigmoid(edf.VDot(edf.ConCat(xt, h), Wr))
    h_hat = edf.Tanh(edf.VDot(edf.ConCat(xt, edf.Mul(r, h)), W))
    h_next = edf.Add(edf.Mul(z, h_hat), edf.Mul(h, edf.Add(edf.Value(1), edf.Mul(z, edf.Value(-1)))))
    
    return h_next


# build the model given the input inp, it shoudl return loss and prob
def BuildModel():
    '''
    Global vars:
        inp : Z^{batch_size x seq_size}
        
    '''
    
    #initialize CP
    edf.components = []

    B = inp.value.shape[0] #batch_size
    T = inp.value.shape[1] #length of sequence (max len(s))
    h = edf.Value(np.zeros((B, hidden_dim))) #initial state of the cell
    
    score = []
    
    for t in range(T-1):
 
        wordvec = edf.Embed(edf.Value(inp.value[:,t]), C2V) 
        xt = edf.Reshape(wordvec, [-1, hidden_dim])
        h_next = GRUCell(xt, h)
        p = edf.SoftMax(edf.VDot(h_next, V))
        logloss = edf.Reshape(edf.LogLoss(edf.Aref(p, edf.Value(inp.value[:,t+1]))), (B, 1))
        
        if t == 0:
            loss = logloss
        else:
            loss = edf.ConCat(loss, logloss)
            
        score.append(p)    
        h = h_next 
    
    masks = np.zeros((B, T-1), dtype = np.int32)
    masks[inp.value[:,1:] != 0] = 1
    loss = edf.MeanwithMask(loss, edf.Value(masks)) 
    
    return loss, score
    

# calculate the perplexity     
def CalPerp(score):
    
    prob = [p.value for p in score]
    prob = np.transpose(np.stack(prob, axis = 0),(1,0,2))
    
    B = prob.shape[0]
    T = prob.shape[1]
    V = prob.shape[2]
    
    masks = np.zeros((B, T), dtype=np.int32)
    masks[inp.value[:,1:] != 0] = 1
    
    prob = prob.reshape(-1)
    idx = np.int32(inp.value[:,1:].reshape(-1))
    outer_dim = len(idx)
    inner_dim = len(prob)/outer_dim
    pick = np.int32(np.array(range(outer_dim))*inner_dim + idx)
    prob = prob[pick].reshape(B, T)
        
    return -np.sum(np.log(prob[np.nonzero(prob*masks)]))


# predict the sequence
def Predict(max_step, prefix):
   
    edf.components = []

    T = max_step       
    h = edf.Value(np.zeros((1, hidden_dim))) 
    prediction = []

    for t in range(T):
   
        if t < len(prefix):
            pred = edf.Value(prefix[t])
            prediction.append(pred)              
        else:
            prediction.append(pred)

        wordvec = edf.Embed(pred, C2V)
        xt = edf.Reshape(wordvec, [-1, hidden_dim])
        h_next = GRUCell(xt, h)
        p = edf.SoftMax(edf.VDot(h_next, V))
        pred = edf.ArgMax(p)
        h = h_next
           
            
    edf.Forward()
    
    idx = [pred.value for pred in prediction]
    stop_idx = utils.to_index('}')
    
    if stop_idx in idx:
        return idx[0:idx.index(stop_idx)+1]
    else:
        return idx  

    
def Eval(data, cnt):
    
    perp = 0.
    avg_loss = 0.
    test_batches = range(0, len(data), batch)
    test_minbatches = [data[idx:idx+batch] for idx in test_batches]
    
    for minbatch in test_minbatches:
        
        x_padded = utils.make_mask(minbatch)
        inp.set(x_padded)
        loss, score = BuildModel()
        edf.Forward()
        avg_loss += loss.value
        perp += CalPerp(score)
           
    perp = np.exp(perp/cnt)
    avg_loss /= len(test_batches)
    return perp, avg_loss


############################################### training loop #####################################################

batches = range(0, len(train_data), batch)
minbatches = [train_data[idx:idx+batch] for idx in batches]

epoch = 30

# initial Perplexity and loss
perp, loss = Eval(valid_data, vacnt)
print("Initial: Perplexity: %0.5f Avg loss = %0.5f" % (perp, loss))    
best_loss = loss
prefix = 'the agreements bring'  
generation = Predict(400, utils.to_idxs(prefix))
print("Initial generated sentence ")
print (utils.to_string(generation))

for ep in range(epoch):

    perm = np.random.permutation(len(minbatches)).tolist() 
    stime=time()
    
    for k in range(len(minbatches)):
        
        minbatch = minbatches[perm[k]]
        x_padded = utils.make_mask(minbatch)
        inp.set(x_padded)
        loss, score = BuildModel()
        edf.Forward()
        edf.Backward(loss)
        edf.GradClip(10)
        edf.SGD(eta)
       
    duration = (time() - stime)/60.
    
    perp, loss = Eval(valid_data, vacnt)
    print("Epoch %d: Perplexity: %0.5f Avg loss = %0.5f [%.3f mins]" % (ep, perp, loss, duration))
    
    # generate some text given the prefix and trained model
    prefix = 'the agreements bring'  
    generation = Predict(400, utils.to_idxs(prefix))
    print("Epoch %d: generated sentence " % ep)
    print (utils.to_string(generation)) 

    if loss < best_loss:
        
        best_loss = loss
        # save the model
        f = open(model, 'wb')
        p_value = []
        for p in parameters:
            p_value.append(p.value)
        pickle.dump(p_value, f)
        
    else:
        
        # load the last best model and decay the learning rate
        eta *= decay
        with open(model, 'rb') as f:
            p_value = pickle.load(f)
            idx = 0
            for p in p_value:
                parameters[idx].value = p
                idx += 1

KeyboardInterrupt: 